# wtf is a metaclass

recommended that you do not use them - overly complex and result in bad code most of the time

a class defines the rules for an object- what parameters, attributes, methods, operations can be performed.

a meta class defines the rules for a class.

when you create a class, this is happening automatically

In [1]:
class Test:
    pass

print(Test)

print(Test())
# our class has type __main__.Test
print(type(Test()))

#but what is the type of a class?

print(type(Test))
#our class is type type - this is the metaclass. this is what is defining our rules, and creates our class.
#when we define our class, we are calling the type constructor that makes our class object, that can then be used to 
#make instances of the object

#for example we can use type to create the class
#this is exactly the same as above
Test = type('Test', (), {})

#the format is Name, bases (anything we want to inherit from, any base classes or super classes, and then any {attributes})
print(Test())

# e.g. we can set an attribute x to 5
Test = type('Test', (), {'x': 5})
print(Test().x)


<class '__main__.Test'>
<class '__main__.Test'>
<class 'type'>
5


In [2]:
class Foo:
    def show(self):
        print('hi')
        
def add_attribute(self):
    self.z = 9
    
Test = type('Test', (Foo,), {'x': 5, 'add_attribute': add_attribute})

t = Test()
t.show()
t.add_attribute()
print(t.z)

#All we are really doing when we define our class is we are passing the information onto type, which is creating an
# object of that class

hi
9


In [3]:
# what about meta classes?
# metaclass is above the class that you are defining. when you define your class, that information is 
# passed up to the metaclass, which is then creating the class object.
# so instead of using our built in type class,we can make our own metaclass - this inherits from type

class Meta(type):
    # this is the first thing that is always called, even before the init. 
    #the init is just taking the already created object, and modifies it by setting values etc
    def __new__(self, class_name, bases, attrs):
        print(attrs)
        return type(class_name, bases, attrs)
        
    
class Dog(metaclass = Meta):
    x = 5
    y = 8
    
    def hello(self):
        print('hi')
    
#d = Dog()

#we have overridden the metaclass with our own one
# we can see our print call

{'__module__': '__main__', '__qualname__': 'Dog', 'x': 5, 'y': 8, 'hello': <function Dog.hello at 0x00000171A90438B8>}


In [4]:
# maybe we want to change everything to be upper case

class Meta(type):
    def __new__(self, class_name, bases, attrs):
        print(attrs)
        a = {}
        
        for item, val in attrs.items():
            if item.startswith('__'):
                a[item] = val
            else:
                a[item.upper()] = val
        print(a)
        return type(class_name, bases, a)
        
    
class Dog(metaclass = Meta):
    x = 5
    y = 8
    
    def hello(self):
        print('hi')
    

{'__module__': '__main__', '__qualname__': 'Dog', 'x': 5, 'y': 8, 'hello': <function Dog.hello at 0x00000171A9043798>}
{'__module__': '__main__', '__qualname__': 'Dog', 'X': 5, 'Y': 8, 'HELLO': <function Dog.hello at 0x00000171A9043798>}


In [5]:
d = Dog()
print(d.x)

AttributeError: 'Dog' object has no attribute 'x'

In [6]:
d.HELLO()

hi


In [7]:
print(d.X)

5


for example we can use it to force a class to be constructed with a certain method, or whatever. 

you can inherit a metaclass in a metaclass

so in the below example, we are creating a Singleton metaclass, which will be used to create our Class object
we can see that it is not overriding the \_\_new\_\_ method - so this will be the same

Instead we are overriding the \_\_call\_\_ method

so here our meta class will remember if we have already created an instance of a class. if we have not, then 
the meta \_\_call\_\_ is going to super our cls, which will  run the \_\_new\_\_ and \_\_init\_\_ of the cls.

see the below example with new instead, to see why that would not work (basically because the \_\_new\_\_ is called when our class is created, not when we call the created class to make an instance of it)

In [8]:
class Singleton(type):
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        print(cls._instances)
        if cls not in cls._instances:
            # we want to excecute the __call__ method of the class that is the level above Singleton in the inheritence
            #in this case that means we call type.__new__() which executes the class __new__ and __init__ by default
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        else:
            print("""we already see that our metacalss made an instance of the class- in this case, we are just wanting
                  to return our instance""")
        return cls._instances[cls]
    

class Database(metaclass = Singleton):
    def __init__(self):
        print('executing Database.__init__()')
        print('loading db')
    
    #added for clarity
    def __new__(cls, *a, **kw):
        print('executing Database.__new__()')
        rv = super(Database, cls).__new__(cls, *a, **kw)
        print('done with __new__')
        return rv

In [9]:
d1 = Database()


{}
executing Database.__new__()
done with __new__
executing Database.__init__()
loading db


In [10]:
d2 = Database()

{<class '__main__.Database'>: <__main__.Database object at 0x00000171A8F8FF88>}
we already see that our metacalss made an instance of the class- in this case, we are just wanting
                  to return our instance


In [11]:
#THIS DOES NOT WORK!! 
#we can see that the __new__ is being called when we specify the class, NOT WHEN WE instanciate it!!!!

# This is why we must use the call() instead. we want it to be executed when we call our class definition, not when we
#create the class object

class Singleton(type):
    _instances = {}
    
    def __new__(cls, *args, **kwargs):
        print(cls._instances)
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__new__(cls, *args, **kwargs)
        else:
            print("""we already see that our metacalss made an instance of the class- in this case, we are just wanting
                  to return our instance""")
        return cls._instances[cls]
    

class Database(metaclass = Singleton):
    def __init__(self):
        print('executing Database.__init__()')
        print('loading db')
    
    #added for clarity
    def __new__(cls, *a, **kw):
        print('executing Database.__new__()')
        rv = super(Database, cls).__new__(cls, *a, **kw)
        print('done with __new__')
        return rv

{}


# my attempt at a meta class

this metaclass inspects annotations and enforces strong typing for them when running a class init

In [12]:
import inspect

class Typed(type):
    """meta class that checks provided arguments against annotations, and enforces typing. 
    raises TypeError if arg is found with incorrect type"""
    def __call__(cls, *args, **kwargs):
        cls_ = super(Typed, cls).__call__(*args, **kwargs)
        expected_args = inspect.signature(cls_.__init__)        
        params = list(expected_args.parameters.values())
        # check the args
        for i,arg in enumerate(args):
            param = params[i]
            Typed._check_param_arg(param, arg)
            
        #check the kwargs
        for a,b in kwargs.items():
            param = expected_args.parameters[a]
            Typed._check_param_arg(param, b) 
        return cls_    
    
    def _check_param_arg(param, arg):
        if param.annotation == param.empty:
            pass
        elif param.annotation != type(arg):
            raise TypeError(' argument with value {} must have type {}'.format(arg, param.annotation))
        else:
            pass
        return None

In [13]:
class EG(metaclass = Typed):
    def __init__(self, a, b: str, c: float, d = 'a'):
        self.a = a

In [14]:
e = EG(1,b = 'f', c= 1.2) #works

In [15]:
e = EG(1,b = 'f', c= "1.2") #error

TypeError:  argument with value 1.2 must have type <class 'float'>

In [16]:
# supports sub classing
class f(EG):
    pass

In [17]:
g = f(1,2,3)

TypeError:  argument with value 2 must have type <class 'str'>